## Purpose

This homework is designed to give you practice with scikitlearn.  Please note that this is **NOT** a machine learning course.  Using the library the important part, not designing 'good' models.  The requirements are fairly low on this.

## Requirements

This is a group assignment.  Take a data set (either one provided, or using your group project data set) and work with Scikit Learn to train some aspect of your data set.

Some data sets may appear to be something you wouldn't use ML to solve in a 'real life' situation, but this again is just for practice.  So the models may not come out useful, and that's okay.

Each student in the group should do 2 ML type implementations using Scikit learn.  Since there are likely less applicable algorithms than there are implementations, work at looking at different slices of information (See help video).


## Required Hand-in

One notebook should be handed in.  Following best practices I've outlined.  This homework is graded as a group homework.  The data set you pick to do this practice can be either one I'm providing as part of the repo, or of your group project.

Please label each implementation with the original author (in code, comment above the implementation).

Do not use the .todo as your template.  Analysis of the models performance should be minimal (see one example on block 10 on https://github.com/TheDarkTrumpet/BAIS-6040-0EXP-Sum2021/blob/master/Notebooks/02-Analysis/09.03.01-Classification.ipynb ).

I do recommend that you lean on whoever in your group has a bit more knowledge of ML concepts. to pick the implementation that appears to yield the best results.  If you're using your group data set, this implementation can then be copied/pasted into the group project.

## Other notes

This homework will be graded as a group.  Meaning, you all will get the same grade, regardless if a specific student's implementation is poorly done.  It will count for 75 points.  I strongly recommend you discuss as a group who will do what, then meet up at least a few days before the assignment is to be turned in and do a code review and merge of the individual notebooks.

In [1]:
import pandas as pd
import numpy as np
import random as rnd
import math
from pytrends.request import TrendReq
from seaborn import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import explained_variance_score, mean_absolute_error, r2_score, mean_squared_error
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, recall_score, precision_score, fbeta_score, classification_report

In [2]:
pytrends = TrendReq(hl='en-US', tz=360)

#build list of keywords in this case only use Teslas
kw_list = ["Bitcoin"] 

# build the payload
pytrends.build_payload(kw_list, timeframe='2021-03-31 2021-06-29', geo='US')

# store interest over time information in df and rename Tesla column to Search Interest
bitcoinTrendsdf = pytrends.interest_over_time()
bitcoinTrendsdf = bitcoinTrendsdf.rename(columns={'Bitcoin': 'Previous_Search_Interest'})
#telsaStockdf.set_index('date': 'Date', inplace=True)
bitcoinTrendsdf.reset_index(inplace=True, drop=True)
bitcoinTrendsdf

,Previous_Search_Interest,isPartial
0,29,False
1,41,False
2,29,False
3,22,False
4,22,False
...,...,...
86,24,False
87,22,False
88,22,False
89,25,False


In [3]:
bitcoinPricedf = pd.read_csv("https://raw.githubusercontent.com/atlas125gev/StockProject/main/Homework7/BTC-USD.csv")
bitcoinPreviousPricedf = pd.read_csv("https://raw.githubusercontent.com/atlas125gev/StockProject/main/Homework7/BTC-USD-Previous.csv")
mergedPrice = pd.concat([bitcoinPricedf, bitcoinPreviousPricedf], axis=1)
bitcoinPreviousPricedf

,Previous_Open,Previous_High,Previous_Low,Previous_Close,Previous_Adj_Close,Previous_Volume
0,58930.277344,59930.027344,57726.417969,58918.832031,58918.832031,65520826225
1,58926.562500,59586.070313,58505.277344,59095.808594,59095.808594,61669163792
2,59098.878906,60267.187500,58869.281250,59384.312500,59384.312500,58727860620
3,59397.410156,60110.269531,57603.890625,57603.890625,57603.890625,59641344484
4,57604.839844,58913.746094,57168.675781,58758.554688,58758.554688,50749662970
...,...,...,...,...,...,...
86,34659.105469,35487.246094,31350.884766,31637.779297,31637.779297,40230904226
87,31594.664063,32637.587891,30184.501953,32186.277344,32186.277344,38585385521
88,32287.523438,34656.128906,32071.757813,34649.644531,34649.644531,35511640894
89,34679.121094,35219.890625,33902.074219,34434.335938,34434.335938,33892523752


In [4]:
mergedStockPrice = pd.concat([mergedPrice, bitcoinTrendsdf], axis=1)
#mergedStockPrice.reset_index(inplace=True, drop=True)
mergedStockPrice.set_index('isPartial', drop=True)
#mergedStockPrice['Previous_Close'] = mergedStockPrice['Close']
#meanSearchInterest = mergedStockPrice['Search_Interest'].mean()
#mergedStockPrice["Interest Points Away From Mean"] = mergedStockPrice["Search Interest"] - meanSearchInterest
mergedStockPrice["Price_Increase"] = mergedStockPrice["Open"] - mergedStockPrice["Close"] > 0.0
mergedStockPrice["Price_Increase"] = mergedStockPrice["Price_Increase"]*1


#pd.set_option('display.max_rows', len(mergedStockPrice))

mergedStockPrice

,Date,Open,High,Low,Close,Adj Close,Volume,Previous_Open,Previous_High,Previous_Low,Previous_Close,Previous_Adj_Close,Previous_Volume,Previous_Search_Interest,isPartial,Price_Increase
0,2021-04-01,58926.562500,59586.070313,58505.277344,59095.808594,59095.808594,61669163792,58930.277344,59930.027344,57726.417969,58918.832031,58918.832031,65520826225,29,False,0
1,2021-04-02,59098.878906,60267.187500,58869.281250,59384.312500,59384.312500,58727860620,58926.562500,59586.070313,58505.277344,59095.808594,59095.808594,61669163792,41,False,0
2,2021-04-03,59397.410156,60110.269531,57603.890625,57603.890625,57603.890625,59641344484,59098.878906,60267.187500,58869.281250,59384.312500,59384.312500,58727860620,29,False,1
3,2021-04-04,57604.839844,58913.746094,57168.675781,58758.554688,58758.554688,50749662970,59397.410156,60110.269531,57603.890625,57603.890625,57603.890625,59641344484,22,False,0
4,2021-04-05,58760.875000,59891.296875,57694.824219,59057.878906,59057.878906,60706272115,57604.839844,58913.746094,57168.675781,58758.554688,58758.554688,50749662970,22,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,2021-06-26,31594.664063,32637.587891,30184.501953,32186.277344,32186.277344,38585385521,34659.105469,35487.246094,31350.884766,31637.779297,31637.779297,40230904226,24,False,0
87,2021-06-27,32287.523438,34656.128906,32071.757813,34649.644531,34649.644531,35511640894,31594.664063,32637.587891,30184.501953,32186.277344,32186.277344,38585385521,22,False,0
88,2021-06-28,34679.121094,35219.890625,33902.074219,34434.335938,34434.335938,33892523752,32287.523438,34656.128906,32071.757813,34649.644531,34649.644531,35511640894,22,False,1
89,2021-06-29,34475.558594,36542.109375,34252.484375,35867.777344,35867.777344,37901460044,34679.121094,35219.890625,33902.074219,34434.335938,34434.335938,33892523752,25,False,0


In [5]:
mergedStockPrice.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91 entries, 0 to 90
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Date                      91 non-null     object 
 1   Open                      91 non-null     float64
 2   High                      91 non-null     float64
 3   Low                       91 non-null     float64
 4   Close                     91 non-null     float64
 5   Adj Close                 91 non-null     float64
 6   Volume                    91 non-null     int64  
 7   Previous_Open             91 non-null     float64
 8   Previous_High             91 non-null     float64
 9   Previous_Low              91 non-null     float64
 10  Previous_Close            91 non-null     float64
 11  Previous_Adj_Close        91 non-null     float64
 12  Previous_Volume           91 non-null     int64  
 13  Previous_Search_Interest  91 non-null     int64  
 14  isPartial   

In [6]:
#columns = ["Open", "High", "Low", "Close", "Search Interest", "Interest Points Away From Mean", "Price Increase Points", "Price Increase"]
columns = ["Previous_Open", "Previous_High", "Previous_Low", "Previous_Close", "Previous_Volume", "Previous_Search_Interest", "Close"]
mergedStockPrice = mergedStockPrice[columns]
mergedStockPrice

,Previous_Open,Previous_High,Previous_Low,Previous_Close,Previous_Volume,Previous_Search_Interest,Close
0,58930.277344,59930.027344,57726.417969,58918.832031,65520826225,29,59095.808594
1,58926.562500,59586.070313,58505.277344,59095.808594,61669163792,41,59384.312500
2,59098.878906,60267.187500,58869.281250,59384.312500,58727860620,29,57603.890625
3,59397.410156,60110.269531,57603.890625,57603.890625,59641344484,22,58758.554688
4,57604.839844,58913.746094,57168.675781,58758.554688,50749662970,22,59057.878906
...,...,...,...,...,...,...,...
86,34659.105469,35487.246094,31350.884766,31637.779297,40230904226,24,32186.277344
87,31594.664063,32637.587891,30184.501953,32186.277344,38585385521,22,34649.644531
88,32287.523438,34656.128906,32071.757813,34649.644531,35511640894,22,34434.335938
89,34679.121094,35219.890625,33902.074219,34434.335938,33892523752,25,35867.777344


In [7]:
features = list(mergedStockPrice.columns)
features.remove("Close")
target = "Close"

X = mergedStockPrice[features]
y = mergedStockPrice[target]

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [9]:
lr = LinearRegression()
lr

LinearRegression()

In [10]:
lr.fit(X_train, y_train)

LinearRegression()

In [11]:
lr.score(X_train, y_train)

0.9600860921909471

In [12]:
lr.score(X_test, y_test)

0.9614613962910867

In [13]:

def printMetrics(test, predictions):
    print(f"Score: {explained_variance_score(test, predictions):.2f}")
    print(f"MAE: {mean_absolute_error(test, predictions):.2f}")
    print(f"RMSE: {math.sqrt(mean_squared_error(test, predictions)):.2f}")
    print(f"r2: {r2_score(test, predictions):.2f}")

In [14]:
predictions = lr.predict(X_test)
printMetrics(y_test, predictions)

Score: 0.96
MAE: 1573.91
RMSE: 1988.37
r2: 0.96


In [15]:
numElements = 3
samplePrice = []
for _ in range(numElements):
    dict = {}
    for column in X.columns:
        min = 0  # We'll always allow at lea
        maxValue = round(max(mergedStockPrice[column].values))
        dict[column] = rnd.randint(min, maxValue)
    samplePrice.append(dict)
samplePrice


[{'Previous_Open': 28486,
  'Previous_High': 6087,
  'Previous_Low': 53661,
  'Previous_Close': 48580,
  'Previous_Volume': 4023584331,
  'Previous_Search_Interest': 31},
 {'Previous_Open': 46111,
  'Previous_High': 47357,
  'Previous_Low': 34499,
  'Previous_Close': 18976,
  'Previous_Volume': 101873542861,
  'Previous_Search_Interest': 28},
 {'Previous_Open': 57766,
  'Previous_High': 41571,
  'Previous_Low': 38142,
  'Previous_Close': 29433,
  'Previous_Volume': 29150590722,
  'Previous_Search_Interest': 88}]

In [16]:
pdSamplePrice = pd.DataFrame.from_dict(samplePrice)
pdSamplePrice


,Previous_Open,Previous_High,Previous_Low,Previous_Close,Previous_Volume,Previous_Search_Interest
0,28486,6087,53661,48580,4023584331,31
1,46111,47357,34499,18976,101873542861,28
2,57766,41571,38142,29433,29150590722,88


In [17]:
predictions = lr.predict(pdSamplePrice)
predictions



array([32559.4987387 , 16419.6841312 , 35193.40838174])

In [18]:
pdSamplePrice = pdSamplePrice.copy()
pdSamplePrice['Predicted'] = predictions
pdSamplePrice



,Previous_Open,Previous_High,Previous_Low,Previous_Close,Previous_Volume,Previous_Search_Interest,Predicted
0,28486,6087,53661,48580,4023584331,31,32559.498739
1,46111,47357,34499,18976,101873542861,28,16419.684131
2,57766,41571,38142,29433,29150590722,88,35193.408382


# Logarithmic Regression

In [19]:
def printClassificationMetrics(test, predictions):
    print("Confusion Matrix:")
    print(confusion_matrix(test, predictions))
    print("------------------")
    print(f"Accuracy: {accuracy_score(test, predictions):.2f}")
    print(f"Recall: {recall_score(test, predictions):.2f}")
    print(f"Prediction: {precision_score(test, predictions):.2f}")
    print(f"f-measure: {fbeta_score(test, predictions, beta=1):.2f}")
    print("------------------")
    print(classification_report(test, predictions))

In [36]:
pd.set_option('display.max_rows', len(mergedStockPrice))

mergedStockPrice["Price_Increase"] = bitcoinPricedf["Open"] - bitcoinPricedf["Close"] > 0.0
mergedStockPrice['Price_Increase'] = mergedStockPrice.Price_Increase.astype(int)
mergedStockPrice


<ipython-input-36-06a67e4b25ec>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mergedStockPrice["Price_Increase"] = bitcoinPricedf["Open"] - bitcoinPricedf["Close"] > 0.0
<ipython-input-36-06a67e4b25ec>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mergedStockPrice['Price_Increase'] = mergedStockPrice.Price_Increase.astype(int)


,Previous_Open,Previous_Close,Previous_Search_Interest,Price_Increase
0,58930.277344,58918.832031,29,0
1,58926.562500,59095.808594,41,0
2,59098.878906,59384.312500,29,1
3,59397.410156,57603.890625,22,0
4,57604.839844,58758.554688,22,0
5,58760.875000,59057.878906,23,1
6,59171.933594,58192.359375,24,1
7,58186.507813,56048.937500,23,0
8,56099.914063,58323.953125,24,1
9,58326.562500,58245.003906,22,0


In [21]:

columns = ["Previous_Open", "Previous_Close", "Previous_Search_Interest", "Price_Increase"]
mergedStockPrice = mergedStockPrice[columns]
mergedStockPrice

features = list(mergedStockPrice.columns)
features.remove("Price_Increase")
target = "Price_Increase"

X = mergedStockPrice[features]
y = mergedStockPrice[target]

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [23]:


logReg = LogisticRegression(solver="liblinear")
logReg

LogisticRegression(solver='liblinear')

In [24]:
logReg.fit(X_train, y_train)

LogisticRegression(solver='liblinear')

In [25]:
logReg.score(X_train, y_train)

0.5294117647058824

In [26]:
logReg.score(X_test, y_test)

0.5652173913043478

In [27]:
predictions = logReg.predict(X_test)
printMetrics(y_test, predictions)

Score: -0.70
MAE: 0.43
RMSE: 0.66
r2: -0.83


In [28]:
predictions = logReg.predict(X_test)
printClassificationMetrics(y_test, predictions)



Confusion Matrix:
[[ 2  7]
 [ 3 11]]
------------------
Accuracy: 0.57
Recall: 0.79
Prediction: 0.61
f-measure: 0.69
------------------
              precision    recall  f1-score   support

           0       0.40      0.22      0.29         9
           1       0.61      0.79      0.69        14

    accuracy                           0.57        23
   macro avg       0.51      0.50      0.49        23
weighted avg       0.53      0.57      0.53        23



In [29]:
pdSamplePrice = pd.DataFrame.from_dict(pdSamplePrice)

predictions = logReg.predict(pdSamplePrice)

pdPredicted = pdSamplePrice
pdPredicted["Price_Increase"] = predictions.astype(bool)
pdPredicted

ValueError: X has 7 features per sample; expecting 3